In [2]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd
from math import radians, degrees, atan2, sqrt

In [3]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [ ]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    
    data: DataFrame
    Mouse moving location, by pixel
    
    !!! We are assuming that the mouse at least reaches the boundary once
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



def dataModifier(data):
    '''
    Task: Modify the location data based on the cropped video
    
    data: DataFrame
    '''
    
    x, y, lengthX, lengthY = getCoord(data)
    filtered = data.copy(deep = True)
    filtered = filtered[(filtered['y']>x) & (filtered['x']>y)]
    filtered['x'] = filtered['x'] - y
    filtered['y'] = filtered['y'] - x
    filtered['coords'] = list(range(1,len(filtered)+1))
    filtered.index = list(range(1,len(filtered)+1))
    
    return filtered

In [ ]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


In [14]:
def quadrant(a, b): 
    '''
    Task: Check if two points are in the same quadrant
    '''
    if ((a[0] > 0 and b[0] > 0) or (a[0] < 0 and b[0] < 0)) and ((a[1] > 0 and b[1] > 0) or (a[1] < 0 and b[1] < 0)):
            return True
    else:
        return False
    
def getAngle(a, b, c):
    '''
    Task: Calculate angle between three points. 
    
    PARAMETERS
    -----------
    a, b, c: tuple
        Three points in the coordinate system
    '''
    cb = atan2(c[1]-b[1], c[0]-b[0])
    ab = atan2(a[1]-b[1], a[0]-b[0])

    # If they are in the same quadrant: acute angle
    if quadrant(c, b) and quadrant(a, b):
        angle = degrees(cb - ab)
    else:
    # Obtuse angle or in different quadrant
        rad360 = radians(360)
        cb = rad360 + cb if cb < 0 else cb
        ab = rad360 + ab if ab < 0 else ab
        angle = abs(degrees(cb - ab))
    return angle + 360 if angle < 0 else angle

def getDist(a, b):
    '''
    Task: Calculate distance
    '''
    dist = sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)  
    return dist  

In [20]:
def calDist_Angle(data, enclosure):
    '''
    Task: Calculate distance and angle between enclosure and defeated mouse
    reference point is the center point of head of tail regarding Angle
    head point for Distance
    
    Parameters
    ----------
    data: DataFrame
        Read location data
    
    enclosure: tuple
        enclosure's coordinate, ex. (68,204)
    '''
    head = pd.concat([data["x"], data["y"]],axis=1)
    tail = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    # Head--------------------Tail-----------
    mid = (head.x + tail.x) / 2, (head.y + tail.y) / 2
    # Avoid Pandas KeyError
    mid = np.array(mid)
    x = np.array(head.x)
    y = np.array(head.y)
    
    angle = []
    dist = []
    for i in range(len(head)):
        # Set middle point of the mouse to be (0,0), then calculate the corresponding relative converted coordinates
        # Order: head relative location, central point, enclosure relative location
        angle.append(getAngle((x[i] - mid[0][i], y[i] - mid[1][i]),(0, 0), (enclosure[0] - mid[0][i], enclosure[1] - mid[1][i])))
        dist.append(getDist((mid[0][i], mid[1][i]), enclosure))
    return angle, dist

In [21]:
for i, value in enumerate(calDist_Angle(data, (68,204))[1]):
    print(value)

167.2214156017847
174.48073540899324
62.173143348216335
62.21581721227461
188.34157501588126
174.12307321451118
61.78030595809677
61.82615030551875
58.45365726938306
61.703553728656914
175.34821304086418
62.0503189168143
61.74578653298651
61.61131960605885
152.22107849976436
152.33988075789824
242.45086270444773
148.8515855136283
173.45026768791433
61.29755208509193
60.844675867325044
174.2014485606727
287.01689821085074
138.4970990193231
138.62503986474593
138.74788340394065
59.282079910952184
61.34180141113543
264.3067424072685
132.85412503265013
132.55366641771082
61.21379214123172
132.13377722499897
132.4657956147614
60.861302441856196
125.19623602102992
128.48727324629098
130.68327164865525
252.40204460479694
130.5503257999093
124.45118649915969
124.12169344326199
125.13550994236971
128.3427251906352
132.21314834332136
128.51302256441295
138.44770977060696
128.3932918159165
192.15102836644064
131.99138740757326
220.2144176002671
131.08385315889427
192.56197953923424
131.8151668792

98.2951187952705
100.79365264482364
98.62565888864827
95.9901422226839
95.87235258914762
99.71002821088025
104.11797518199893
114.67334440011737
120.95746507076394
125.97074310338192
134.96228539416853
144.17829748534504
155.21878580811577
165.31437359922225
175.11261452899865
186.30982531637997
197.48871920865733
205.388868697081
217.5438757061749
230.16990505086463
239.23003419392523
244.02652722848742
250.8869058658741
259.6555182329398
268.2857257753495
272.01818743471586
279.04253520367996
281.6717314448355
282.4697968834129
281.93845286314547
280.2448445290993
269.224521386368
270.50039401550407
269.5933389985036
271.4600043909161
272.57483347177333
279.1108891674643
279.31072574590553
279.37597592687365
281.12818761564523
283.6165618695299
286.1140096762296
289.3556378919316
293.48726056057643
291.1672580526638
290.45658686360315
290.90778171478195
283.37268588324844
282.106930121029
284.35270912999124
285.79489624188756
289.90910440537124
292.01906311420765
294.4873736084856
29

140.48373056372358
140.21590667273222
140.72332303452126
139.72796602890546
144.93678692450132
148.01328011365635
149.80690460610865
149.35085198767393
148.20452767301433
144.57656857805722
145.62811977608973
148.32607559444318
147.4663374352249
145.5458176073734
142.72941816004874
140.8154912158033
140.8813602680719
138.18099759373246
134.524849375984
134.36900618752816
134.69335719036928
132.49449504274835
126.73751055088795
126.16806955634218
122.33042222733476
119.03304353281247
110.4732615195417
106.17114022773491
103.76842541253929
101.38825301699514
102.73368691496509
97.41135987588702
89.91422654737458
86.72841697170415
79.28605724600365
74.81196560709016
74.00553403271523
74.59220580836869
73.90025213507784
73.21127595834263
73.10875106849328
76.72042018769541
77.18134856825895
75.6379108532448
73.93274764119448
73.95617866563266
77.82667080782201
75.21816184043504
74.80187949484231
73.68772476813444
73.15722100316322
71.8703991805485
73.09343086370936
72.98966682535455
74.210

In [4]:
import numpy as np
import cv2
 
def orderPoints(pts):
    '''
    Task: sort points by the following order: top-left, top-right, bottom-left, bottom-left
    
    PARAMETERS:
    -----------
    pts: list 
        list of tuple of points coordinate
    '''
    
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def fourPointTransform(image, pts):
    '''
    Task: Transform image by its corner four points. 
    
    PARAMETERS:
    -----------
    image: array
        image matrix
    
    pts: list
        list of tuple of four corner points
    '''
    rect = orderPoints(pts)
    (tl, tr, br, bl) = rect

    # Transformed width
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # Transformed height
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # Specify the destination points based on the dimensions: top-left, top-right, bottom-right, and bottom-left
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return warped, M



In [1]:
def locDataConvert(data, pts):
    '''
    Task: Convert mouse location data to prospective coordicates after correcting coord system.
    
    PARAMETERS:
    -----------
    pts: list
        list of four corner points of the video
    '''
    # Transformation matrix
    matrix = fourPointTransform(img, pts)[1]
    # x, y, 1
    head = pd.concat([
        data["x"], 
        data["y"], 
        pd.DataFrame([1]*len(data))],axis=1).values.transpose()
    tail = pd.concat([
        data["x.1"], 
        data["y.1"], 
        pd.DataFrame([1]*len(data))], axis = 1, keys=['x', 'y', 'constant']).values.transpose()
    
    transformedHead = np.dot(matrix, head)
    transformedTail = np.dot(matrix, tail)
    
    return transformedHead, transformedTail




In [7]:
pts = np.array([(28,39),(74,387),(364,23),(377,336)])
img = cv2.imread("test2.jpg")
locDataConvert(data,pts)[0]

array([[239.51141475, 253.5844412 , -79.2430577 , ..., 178.49255253,
        176.40512073, 175.91400833],
       [410.02155988, 411.55928776, 385.81028164, ...,  32.66322184,
         32.90061134,  31.19195525],
       [  0.76872823,   0.76286879,   0.89830812, ...,   0.90363667,
          0.90436829,   0.90506166]])

In [11]:
# [i[j] for j in range(locDataConvert(data,pts)[0].shape[1]) for i in locDataConvert(data,pts)[0] ]
start = time()
head = locDataConvert(data,pts)[0]
x = []
y = []
for j in range(head.shape[1]):
    y = []
    for i in head:
        y.append(i[j])
    x.append(y)


0.032510995864868164


[[239.5114147540263, 410.0215598784492, 0.7687282287273324],
 [253.58444120196714, 411.5592877616764, 0.7628687858648031],
 [-79.24305769822806, 385.8102816363677, 0.8983081233627928],
 [-79.45472755845108, 385.60873443300443, 0.8984489592759097],
 [280.20231389672153, 410.91553899509404, 0.7528355049977586],
 [253.52666357389376, 411.02083127926903, 0.7630500380321306],
 [-78.94368057624192, 385.49642611474206, 0.8982858513796199],
 [-79.60916674821604, 385.5528947353368, 0.8985247717125633],
 [-50.38616527143955, 392.5454525402606, 0.8852351318539069],
 [-79.49800571994041, 385.3546075867187, 0.8985406510747538],
 [253.71241397361098, 411.1625086397798, 0.7629368429546073],
 [-79.88241329340676, 385.19229524856144, 0.8987362423945795],
 [-79.4091953412561, 385.5391710287169, 0.8984520203018685],
 [-79.38751755471671, 385.38151346235696, 0.8984902665272867],
 [-78.83370076381362, 385.4917810462549, 0.8982449822757098],
 [-79.34925498468182, 385.3982129803453, 0.898470637502019],
 [-78

In [ ]:
csvfile

In [ ]:
test = move_data
test

In [ ]:
import numpy as np
pd.options.mode.chained_assignment = None
p['coords'] = list(range(1,len(p)+1))
p.index = list(range(1,len(p)+1))

In [ ]:
p

In [ ]:
p = test[(test['y']>54) & (test['x']>38)]

In [ ]:
test = move_data['x'] - 38
test[test>0]

In [ ]:
def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out

def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector, millimeter

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees
    

    head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
    tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    degrees = head_dir(data)[1]
    
    #Get pixel to mm conversion
    x, y, lengthX, lengthY = getCoord(data)
    length = lengthX if lengthX < lengthY else lengthY
    convert = 440/length # video side length 440mm, square
    radius = round(radius / convert)
    
    
    #cap: Read video
    #out: Write video
    video = videoWriter("field of view", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        try:
            for center, angle in tqdm(zip(head, degrees)):
                ret,img = cap.read()
            #draw the sector representing the field of view of mouse
                angle = int(round(angle))
                startAngle = angle - viewRange
                endAngle = angle + viewRange
                img2 = drawSector(img, radius, center, rotationAngle, startAngle, endAngle, color, thickness)
                out.write(img2)
                    
                if displayVideo == True:
                    cv2.imshow('Draw field of view video',img2) 
                    
                # Break the loop when the last frame has been rotated
                if count == frameCnt:
                    print (filename, 'successfully ', "added field of view!!!")
                    return
                else:
                    count += 1

                k=cv2.waitKey(30) & 0xff
                #once you inter Esc capturing will stop
                if k==27:
                    break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [ ]:
BLACK = (0,0,0)
from math import degrees, atan2
def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)

    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0]) # End's coordinates
    point2 = tuple(points[-1]) # Another end's coordinates
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image

def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)

In [ ]:
x = dataModifier(move_data)

In [ ]:
fieldOfView(x, '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest1.mp4", 50, 0, BLACK, 60, 1)

In [ ]:
x, y, lengthX, lengthY = getCoord(move_data)

In [ ]:
x, y, lengthX, lengthY

In [ ]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

In [ ]:
440/331

In [ ]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

In [ ]:
##Question: choose length of the edge in order to be converted in video processing